# Data Import

In [2]:
import pandas as pd

# Import our raw data into a Pandas Dataframe
# low_memory is just to make the 'mixed types' warning go away.
df1 = pd.read_csv('delta_public_87_99.csv', low_memory=False)
df2 = pd.read_csv('delta_public_00_12.csv', low_memory=False)

# Combine the 87-99 data with the 00-12 data into one frame
frames = [df1, df2]
data = pd.concat(frames)

## Data Exploration

In [3]:
# Get number of observations and number of attributes
print data.shape

(215613, 974)


In [4]:
# Isolate Target Variable
enrollment = data['total_enrollment']
features = data.drop('total_enrollment', axis = 1)

In [5]:
enrollment.describe()

count    153168.000000
mean       2809.486148
std        7540.511589
min           0.000000
25%         101.000000
50%         481.000000
75%        2344.000000
max      380232.000000
Name: total_enrollment, dtype: float64